In [3]:
# !pip install deeppavlov
# !pip install transformers
# !pip install pymorphy

In [ ]:
from deeppavlov import configs, build_model
ner_model = build_model(configs.ner.ner_rus_bert_torch, download=True)

In [ ]:
article_text = 'РОСТОВ-НА-ДОНУ, ул. Митрофанова д.22к1, 2 фев – РИА Новости. Тело мужчины, сбежавшего более двух месяцев назад из Сарпинского психоневрологического диспансера в Калмыкии, обнаружили в поселке неподалеку, сообщает СУСК России по Калмыкии. По информации следствия, вечером 29 ноября мужчина самовольно покинул психдиспансер. По факту безвестного исчезновения мужчины было возбуждено уголовное дело по статье "Убийство". Был организован его розыск."В дополнение к ранее опубликованной информации о розыске подопечного БУРК "Сарпинский психоневрологический диспансер" Строкова А.Н. региональное следственное управление СК России сообщает, что на территории поселка Годжур Сарпинского района обнаружено тело мужчины", - говорится в сообщении.В настоящее время по данному факту организована проверка.'

In [ ]:
from graph import InferenceWorker
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [17]:
class SampleNER(InferenceWorker):
    def __call__(self, t, d):
        if t == "text":
            yield "entities", self.ner_from_text(d), "NER"
        else:
            yield None        
    def ner_from_text(self, text):
        result = ner_model([text])
        entities = []
        new_entity = None
        new_tag = None
        for entity, tag in zip(result[0][0], result[1][0]):
            if tag == 'O':
                if new_entity != None:
                    entities.append([new_entity, new_tag])
                    new_tag = None
                    new_entity = None     
            else:
                if tag[0] == 'B': 
                    if new_entity != None:
                      entities.append([new_entity, new_tag])
                    new_entity = entity
                    new_tag = tag[2:]
                if tag[0] == 'I':
                  new_entity += (' ' + entity)
        if new_entity != None:
            entities.append([new_entity, new_tag])
        for x in entities:
          if x[1] == 'LOC':
            if x[0].isalpha():
              x[0] = morph.normal_forms(x[0])[0].capitalize()
            else:
              x[0] = x[0].capitalize()
        return entities
        

In [18]:
worker = SampleNER()
worker.serialize()

Worker saved!


In [ ]:
for x in worker("text", article_text):
    print(x)